In [ ]:
import pandas as pd

from data_processing import load_and_process_all_buildings, process_resstock_data

# Load data

## ResStock

In [ ]:
resstock = pd.read_parquet("/Users/adrian/Documents/ICAI/4o/AI Lab/data/MA_baseline_metadata_and_annual_results.parquet")

## Buildings Timeseries

In [ ]:
buildings = load_and_process_all_buildings(resstock)

## Weather data

In [ ]:
weather_data = pd.read_csv("/Users/adrian/Documents/ICAI/4o/AI Lab/data/G2500170_2018.csv")

# Process data

## ResStock

In [ ]:
resstock_processed = process_resstock_data(resstock)

## Weather

In [ ]:
# Change date time column name to timestamp
weather_data.rename(columns={'date_time': "timestamp"}, inplace=True)
weather_data.set_index('timestamp', inplace=True)
weather_data.index = pd.to_datetime(weather_data.index)

In [ ]:
# Delete all columns except dry bulb temperature and direct normal radiation
weather_data = weather_data[["Dry Bulb Temperature [°C]", "Direct Normal Radiation [W/m2]"]]
weather_data.rename(columns={"Dry Bulb Temperature [°C]": "temperature"}, inplace=True)

In [ ]:
# Create an entry for 2018-01-01 00:00:00
new_row = pd.DataFrame({'temperature': weather_data.iloc[0]['temperature']}, index=[pd.to_datetime('2018-01-01 00:00:00')])
weather_data = pd.concat([new_row, weather_data])

In [ ]:
weather_data.bfill(inplace=True)
weather_data.ffill(inplace=True)